# <center>Лабораторная работа №6
## <center>Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина

<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код.**

In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [2]:
data = pd.read_csv('winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [5]:
X = np.array(data.iloc[:,[x for x in range(0, 11)]])
y = np.array(data["quality"])

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.7, random_state=17) # Ваш код здесь
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [6]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [7]:
from sklearn.metrics import mean_squared_error
train_pred = linreg.predict(X_train_scaled)
test_pred = linreg.predict(X_holdout_scaled)
print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.5321779540981326
Mean squared error (test): %.3f 0.5835198458691796


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [8]:
linreg_coef = pd.DataFrame(linreg.coef_, index=data.columns[:11])
linreg_coef.sort_values(0, ascending=False)

,0
residual sugar,0.560499
pH,0.151993
alcohol,0.140476
fixed acidity,0.100363
sulphates,0.065973
total sulfur dioxide,0.033827
citric acid,0.013249
chlorides,0.011563
free sulfur dioxide,-0.010237
volatile acidity,-0.192491


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [9]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [10]:
lasso1_coef = pd.DataFrame(lasso1.coef_, index=data.columns[:11])
lasso1_coef.sort_values(0, ascending=False)

,0
alcohol,0.343034
residual sugar,0.251695
pH,0.063068
sulphates,0.032678
free sulfur dioxide,0.001119
fixed acidity,-0.000000
citric acid,0.000000
chlorides,-0.000000
total sulfur dioxide,0.000000
volatile acidity,-0.189060


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [11]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv='warn', eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [12]:
lasso_cv.alpha_

0.001135733358343105

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина.**<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [13]:
lasso_cv_coef = pd.DataFrame(lasso1.coef_, index=data.columns[:11])
lasso_cv_coef.sort_values(0, ascending=False)

,0
alcohol,0.343034
residual sugar,0.251695
pH,0.063068
sulphates,0.032678
free sulfur dioxide,0.001119
fixed acidity,-0.000000
citric acid,0.000000
chlorides,-0.000000
total sulfur dioxide,0.000000
volatile acidity,-0.189060


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [14]:
train_pred = lasso_cv.predict(X_train_scaled)
test_pred = lasso_cv.predict(X_holdout_scaled)
print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.5323476509094052
Mean squared error (test): %.3f 0.5817357788547326


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [16]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)
print(X_train_scaled.shape)
print(y_train.shape)

(1469, 11)
(1469,)


**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [17]:
train_pred = forest.predict(X_train_scaled)
test_pred = forest.predict(X_holdout_scaled)
cross = abs(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error').mean())
print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (cv): %.3f", cross)
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.08962559564329475
Mean squared error (cv): %.3f 0.5303223710752194
Mean squared error (test): %.3f 0.5304957713619131


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [18]:
from sklearn.ensemble import RandomForestClassifier

forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

rfc = RandomForestClassifier()

locally_best_forest = GridSearchCV(rfc, forest_params)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 17, 'max_features': 7, 'min_samples_leaf': 4},
 0.5793056501021103)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [20]:
forest = RandomForestClassifier(max_depth=19, min_samples_leaf=1, max_features=7, random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=19, max_features=7, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [21]:
train_pred = forest.predict(X_train_scaled)
test_pred = forest.predict(X_holdout_scaled)
print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.02110279101429544
Mean squared error (test): %.3f 0.6587926509186351


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [22]:
rf_importance = pd.DataFrame(forest.feature_importances_, index=data.columns[:11])
rf_importance.sort_values(0, ascending=False)

,0
alcohol,0.138509
pH,0.095619
volatile acidity,0.092148
total sulfur dioxide,0.090930
residual sugar,0.090614
free sulfur dioxide,0.085557
sulphates,0.084620
density,0.083849
citric acid,0.081850
chlorides,0.080494


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**